# Web application

https://pypi.org/project/yfinance/

https://github.com/ranaroussi/yfinance

Short usage instructions for our needs:
`sym = yf.Ticker('YAHOOSYMBOL')` and data retrieval `sym.history(period, interval, auto_adjust=True)`

- period : valid values '1mo', '5d', '1d'
- interval : valid values

In [6]:
!pip install yfinance

  Running setup.py bdist_wheel for yfinance ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
  Running setup.py bdist_wheel for multitasking ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/37/fa/73/d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import yfinance as yf

In [17]:
sym_ixic = yf.Ticker('^IXIC')

In [20]:
sym_ixic.history(period='1mo')['2019-12-31':]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-31,8918.74,8975.36,8912.77,8972.60,2182800000,0,0
2020-01-02,9039.46,9093.43,9010.89,9092.19,2848370000,0,0
2020-01-03,8976.43,9065.76,8976.43,9020.77,2567400000,0,0
2020-01-06,8943.50,9072.41,8943.50,9071.47,2788120000,0,0
2020-01-07,9076.64,9091.93,9042.55,9068.58,2352850000,0,0
2020-01-08,9068.03,9168.89,9059.38,9129.24,2464090000,0,0
2020-01-09,9202.27,9215.95,9158.50,9203.43,2534700000,0,0
2020-01-10,9232.95,9235.20,9164.66,9178.86,2378990000,0,0
2020-01-13,9213.72,9274.49,9193.06,9273.93,2530270000,0,0
